In [ ]:
#!pip install annoy

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# Read the CSV file into a DataFrame
games = pd.read_csv("games.csv")

# Preprocess the data
games['Publishers'] = games['Publishers'].fillna('')
games['Genres'] = games['Genres'].fillna('')
games['Features'] = games['Publishers'] + ' ' + games['Genres']

# Create a TF-IDF vectorizer for both combined features and genres
tfidf_vectorizer_combined = TfidfVectorizer()
tfidf_matrix_combined = tfidf_vectorizer_combined.fit_transform(games['Features'])
tfidf_vectorizer_genres = TfidfVectorizer()
tfidf_matrix_genres = tfidf_vectorizer_genres.fit_transform(games['Genres'])

# Build nearest neighbors models for both combined features and genres
knn_combined = NearestNeighbors(n_neighbors=6, metric='cosine')
knn_genres = NearestNeighbors(n_neighbors=6, metric='cosine')
knn_combined.fit(tfidf_matrix_combined)
knn_genres.fit(tfidf_matrix_genres)

# Function to recommend similar games with "Header image" and "Genres"
def recommend_similar_games(game_name, games_df=games, knn_combined=knn_combined, knn_genres=knn_genres, tfidf_matrix_combined=tfidf_matrix_combined, tfidf_matrix_genres=tfidf_matrix_genres):
    # Find the index of the given game in the DataFrame
    game_index = games_df[games_df['Name'] == game_name].index[0]
    
    # Use the nearest neighbors model to find the most similar games for both combined and genres models
    distances_combined, indices_combined = knn_combined.kneighbors(tfidf_matrix_combined[game_index])
    distances_genres, indices_genres = knn_genres.kneighbors(tfidf_matrix_genres[game_index])
    
    # Exclude the input game itself from recommendations for both models
    similar_game_indices_combined = indices_combined[0][1:]
    similar_game_indices_genres = indices_genres[0][1:]
    
    # Get the 5 most similar games for both models
    similar_games_combined = [games_df.loc[i] for i in similar_game_indices_combined]
    similar_games_genres = [games_df.loc[i] for i in similar_game_indices_genres]
    
    # Check if the priority model (combined) has enough recommendations
    if len(similar_games_combined) >= 5:
        recommended_games = similar_games_combined[:5]
    else:
        # Combine recommendations from the priority model and genres model
        combined_recommendations = similar_games_combined + similar_games_genres
        
        # Remove duplicates and return up to 5 recommendations
        recommended_games = list(set(combined_recommendations))[:5]
    
    return pd.DataFrame(recommended_games)

# Example usage
input_game_name = "Galactic Bowling"
recommended_games_df = recommend_similar_games(input_game_name)

if not recommended_games_df.empty:
    print(f"For the game '{input_game_name}', 5 similar recommendations are:")
    for i, (_, game) in enumerate(recommended_games_df.iterrows()):
        print(f"{i + 1}. Name: {game['Name']}, Header image: {game['Header image']}, Genres: {game['Genres']}")
else:
    print(f"No recommendations found for the game '{input_game_name}'.")


For the game 'Galactic Bowling', 5 similar recommendations are:
1. Name: Football Club Simulator - FCS #21, Header image: https://cdn.akamai.steamstatic.com/steam/apps/324090/header.jpg?t=1622140190, Genres: Casual,Simulation,Sports,Strategy
2. Name: Imperivm RTC - HD Edition Great Battles of Rome, Header image: https://cdn.akamai.steamstatic.com/steam/apps/752580/header.jpg?t=1629132365, Genres: Strategy
3. Name: Arena, Header image: https://cdn.akamai.steamstatic.com/steam/apps/717300/header.jpg?t=1558079788, Genres: Free to Play,Indie,Strategy
4. Name: Time Keeper, Header image: https://cdn.akamai.steamstatic.com/steam/apps/1727500/header.jpg?t=1661385291, Genres: Adventure
5. Name: Dragon Lords: 3D Strategy, Header image: https://cdn.akamai.steamstatic.com/steam/apps/661960/header.jpg?t=1505189671, Genres: Free to Play,Massively Multiplayer,Strategy


In [ ]:
games

In [ ]:
games

In [ ]:
editgames=pd.read_csv("editGame.csv")
editgames